<a href="https://colab.research.google.com/github/HardworkingPearl/VCC-state/blob/evo2emb/Experiments/dataload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# install packages

In [2]:
|!pip install pyreadr
!pip install tzlocal
!pip install anndata
!sudo apt-get update -y || apt-get update -y
!sudo apt-get install -y build-essential gfortran cmake \
  libxml2-dev libcurl4-openssl-dev libssl-dev \
  libomp-dev libgsl-dev

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.0/80.0 kB 6.9 MB/s eta 0:00:00
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubu

## Install to google drive and Read

In [ ]:
R_LIB = "/content/drive/MyDrive/R"

import os
os.makedirs(R_LIB, exist_ok=True)

from rpy2.robjects import r

r(f'''
.libPaths(c("{R_LIB.replace("\\", "/")}", .libPaths()))

Sys.setenv(R_LIBS="{R_LIB.replace("\\", "/")}")
Sys.setenv(R_LIBS_USER="{R_LIB.replace("\\", "/")}")

cat("R library paths:\\n"); print(.libPaths()); cat("\\n")

options(repos=c(CRAN="https://cloud.r-project.org"))

if (!requireNamespace("remotes", quietly=TRUE)) install.packages("remotes", lib=.libPaths()[1])

pkgs <- c("Matrix","Rcpp","RSpectra","uwot","RcppAnnoy","igraph")
to_install <- pkgs[!sapply(pkgs, requireNamespace, quietly=TRUE)]
if (length(to_install)) {{
  install.packages(to_install, lib=.libPaths()[1], Ncpus=parallel::detectCores())
}}

remotes::install_github("cole-trapnell-lab/monocle3",
                        lib=.libPaths()[1],
                        upgrade="never",
                        build_vignettes=FALSE)

suppressPackageStartupMessages(library(monocle3, lib.loc=.libPaths()[1]))
cat("loaded monocle3 from GitHub OK\\n")
''')

In [3]:
R_LIB = "/content/drive/MyDrive/R"

from rpy2.robjects import r
r(f'''
.libPaths(c("{R_LIB.replace("\\", "/")}", .libPaths()))
suppressPackageStartupMessages(library(monocle3, lib.loc=.libPaths()[1]))
cat("monocle3 is ready (loaded from Drive)\\n")
''')

monocle3 is ready (loaded from Drive)


## Install to tmp location

In [ ]:
from rpy2.robjects import r
r('''
options(repos=c(CRAN="https://cloud.r-project.org"))
if (!requireNamespace("remotes", quietly=TRUE)) install.packages("remotes")
# Install dependencies first to get clearer errors
remotes::install_cran(c("Matrix","Rcpp","RSpectra","uwot","RcppAnnoy","igraph"), upgrade="never")
remotes::install_github("cole-trapnell-lab/monocle3", upgrade="never", build_vignettes=FALSE)
suppressPackageStartupMessages(library(monocle3)); cat("loaded monocle3 from GitHub OK\\n")
''')


# Read Data and convert to h5ad file

In [4]:
import pandas as pd
from pathlib import Path
from datetime import datetime
from rpy2.robjects import r, globalenv
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import numpy as np
from scipy.sparse import csc_matrix
import anndata as ad
import os
from tqdm import tqdm

raw_dir = "/content/drive/MyDrive/VCC/datasets"
h5_dir = "/content/drive/MyDrive/VCC/datasets/h5"

In [5]:
def _extract_RDS(cds):
  globalenv['cds'] = cds
  r('M <- SummarizedExperiment::assay(cds, "counts")')
  x   = np.array(r('slot(M, "x")'))
  i   = np.array(r('slot(M, "i")'))
  p   = np.array(r('slot(M, "p")'))
  Dim = np.array(r('dim(M)'))

  X = csc_matrix((x, i, p), shape=(int(Dim[0]), int(Dim[1])))

  # metadata
  with localconverter(pandas2ri.converter):
      obs = r('as.data.frame(SummarizedExperiment::colData(cds))')
      var = r('as.data.frame(SummarizedExperiment::rowData(cds))')

  # gene name list
  with localconverter(pandas2ri.converter):
      obs_names = list(r('as.character(colnames(cds))'))
      var_names = list(r('as.character(rownames(cds))'))

  obs.index = obs_names
  var.index = var_names

  return X, obs, var

## Srivatsm

In [6]:
def readRDS(file_path):
  readRDS = r['readRDS']
  cds = readRDS(file_path)
  return _extract_RDS(cds)

In [7]:
def to_h5ad(raw_path, h5_path):
  rds_files = [f for f in os.listdir(raw_path) if f.lower().endswith('.rds')]
  print(rds_files)
  for rds_file in tqdm(rds_files):
    file_path = f"{raw_path}/{rds_file}"
    X, obs, var = readRDS(file_path)
    if X.shape[1] == obs.shape[0]:
      adata = ad.AnnData(X=X.T, obs=obs, var=var)
    else:
      adata = ad.AnnData(X=X, obs=obs, var=var)
    adata.write_h5ad(f"{h5_path}/{rds_file[:-4]}.h5ad", compression="gzip")

In [ ]:
srivatsam_raw_path = f"{raw_dir}/srivatsam"
srivatsam_h5_dir = f"{h5_dir}/srivatsam"

to_h5ad(srivatsam_raw_path, srivatsam_h5_dir)

['GSM4150376_sciPlex1_cds.RDS', 'GSM4150377_sciPlex2_cds.RDS', 'GSM4150378_sciPlex3_A549_24hrs.RDS', 'GSM4150378_sciPlex3_K562_24hrs.RDS', 'GSM4150379_sciPlex4_hdaci_cds.RDS', 'GSM4150378_sciPlex3_MCF7_24hrs.RDS', 'GSM4150378_sciPlex3_cds_all_cells.RDS']


 86%|████████▌ | 6/7 [07:39<01:51, 111.66s/it]

## mcfaline

In [ ]:
def readRDS_cds(file_path):
  readRDS = r['readRDS']
  base = r
  obj = readRDS(file_path)
  globalenv['.__r_obj__'] = obj

  is_cds = bool(r('inherits(.__r_obj__, "cell_data_set")')[0])
  is_list = bool(r('is.list(.__r_obj__)')[0])

  if is_cds:
      return _extract_RDS(obj)

  if not is_list:
      raise

  names = list(map(str, r('names(.__r_obj__)')))
  results = {}
  for nm in names:
      results[nm] = _extract_RDS(obj.rx2(nm))
  return results

In [ ]:
mcfaline_raw_path = f"{raw_dir}/mcfaline"
mcfaline_h5_dir = f"{h5_dir}/mcfaline"

In [ ]:
rds_files = [f for f in os.listdir(mcfaline_raw_path) if f.lower().endswith('.rds')]
rds_files

In [ ]:
rds_file = ""
file_path = f"{mcfaline_raw_path}/{rds_file}"
readRDS = r['readRDS']
obj = readRDS(file_path)
globalenv['.__r_obj__'] = obj

is_cds = bool(r('inherits(.__r_obj__, "cell_data_set")')[0])
is_list = bool(r('is.list(.__r_obj__)')[0])

### Single dataset

In [ ]:
X, obs, var = readRDS(file_path)
if X.shape[1] == obs.shape[0]:
  adata = ad.AnnData(X=X.T, obs=obs, var=var)
else:
  adata = ad.AnnData(X=X, obs=obs, var=var)
adata.write_h5ad(f"{mcfaline_h5_dir}/{rds_file[:-4]}.h5ad", compression="gzip")

### ListVector

In [ ]:
names = list(map(str, r('names(.__r_obj__)')))
results = {}
for nm in names:
    X, obs, var = readRDS(obj.rx2(nm))
    if X.shape[1] == obs.shape[0]:
      adata = ad.AnnData(X=X.T, obs=obs, var=var)
    else:
      adata = ad.AnnData(X=X, obs=obs, var=var)
    adata.write_h5ad(f"{mcfaline_h5_dir}/{rds_file[:-4]}_{nm}.h5ad", compression="gzip")

### Special Case

In [ ]:
obj

In [ ]:
with localconverter(pandas2ri.converter):
  df = pd.DataFrame(obj)

In [ ]:
df.columns = ["new_cell", "gRNA", "read_count", "umi_count"]
df.to_csv("f"{mcfaline_h5_dir}/{rds_file[:-4]}.txt", sep="\t", index=False)